# Predict ED admission probability

This notebook demonstrates the first stage of prediction, to generate a probability of admission for each patient in the ED. 

As one of the modelling decisions is to send predictions at specified times of day, we tailor the models to these times and train one model for each time. The dataset used for this modelling is derived from snapshots of visits at each time of day. The times of day are define in config.json file in the root directory of this repo. 

A patient episode (visit) may well span more than one of these times, so we need to consider how we will deal with the occurence of multiple snapshots per episode. At each of these times of day, we will use only one training sample from each hospital episode.

Separation of the visits into training, validation and test sets will be done chronologically into a training, validation and test set 

Evaluation of individual level models includes: 
- feature importance plots
- calibration plot
- MADCAP overall, plus breakdown by age category and length of stay


## Set up the notebook environment

In [1]:
# Reload functions every time
%load_ext autoreload 
%autoreload 2

In [7]:
from pathlib import Path
import sys
import json


PROJECT_ROOT = Path().home() / 'HyMind'

# Patient flow package
USER_ROOT = Path().home()
sys.path.append(str(USER_ROOT / 'patientflow' / 'patientflow' ))

# Functions that sit outside the package
sys.path.append(str(USER_ROOT / 'patientflow' / 'functions' ))




In [ ]:
model_file_path = PROJECT_ROOT / 'dissemination' / 'model-output' / 'trained-models'
model_file_path

data_file_path = PROJECT_ROOT / 'dissemination' / 'data-raw'
data_file_path

In [ ]:
sys.path

## Load parameters

These are set in config.json. You can change these for your own purposes. But the times of day will need to match those in the provided dataset if you want to run this notebook successfully.

In [8]:
# Load the times of day
import yaml

config_path = Path(PROJECT_ROOT / 'dissemination')

with open(config_path / 'config.yaml', 'r') as file:
    config = yaml.safe_load(file)
    
# Convert list of times of day at which predictions will be made (currently stored as lists) to list of tuples
prediction_times = [tuple(item) for item in config['prediction_times']]

# See the times of day at which predictions will be made
prediction_times

ModuleNotFoundError: No module named 'yaml'

## Load data

In [ ]:
from ed_admissions_data_retrieval import ed_admissions_get_data
PATH_ED = 'HyMind/dissemination/data-raw/ED_visits.csv'

df = ed_admissions_get_data(PATH_ED)

In [ ]:
df.head()

See how many visits there are at each time of day in the dataset. We see that number of visits represented is greater in the afternoon and evening

In [ ]:
print(df.prediction_time.value_counts())

We will confirm that the dataset aligns with the specified times of day set in the parameters file config.yaml. That is because, later, we will use these times of day to evaluate the predictions. The evaluation will fail if the data loaded does not match. 

In [ ]:
print("\nTimes of day at which predictions will be made")
print(prediction_times)
print("\nNumber of rows in dataset that are not in these times of day")
print(len(df[~df.prediction_time.isin(prediction_times)]))

## Set an index column in df

Setting the index as the snapshot_id before subsetting means that we retain the same values of snapshot_id throughout the entire process, ensuring that they are consistent across the original dataset df and the training, validation and test subsets of df

In [ ]:
df.head()

After executing the code below, the snapshot_id has been set as the index column.

In [ ]:
if df.index.name != 'snapshot_id':
    df = df.set_index('snapshot_id')
df.head()

## Separate into training, validation and test sets

As part of preparing the data, each visit has already been allocated into one of three sets - training, vaidation and test sets. This has been done chronologically, as shown by the output below. Using a chronological approach is appropriate for tasks where the model needs to be validated on unseen, future data.


In [ ]:
for value in df.training_validation_test.unique():
    subset = df[df.training_validation_test == value]
    counts = subset.training_validation_test.value_counts().values[0]
    min_date = subset.snapshot_datetime.min()
    max_date = subset.snapshot_datetime.max()
    print(f"Set: {value}\nNumber of rows: {counts}\nMin Date: {min_date}\nMax Date: {max_date}\n")



In [ ]:
train_df = df[df.training_validation_test == 'train'].drop(columns='training_validation_test')
valid_df = df[df.training_validation_test == 'valid'].drop(columns='training_validation_test')
test_df = df[df.training_validation_test == 'test'].drop(columns='training_validation_test')


We can see below that some visits appear more than once in each of these sets. (No visit appears in more than one set.)

In [ ]:
train_df.visit_number.value_counts()

For example, the below patient has 35 episode slices, of which the majority are in the ED location 'OTF'. OTF refers to 'Off the Floor'. Patients can sometimes be moved to this location when paperwork is due, while in fact the patient has already left the ED. While it is tempting to remove these OTF locations, in real-time these patients would be picked up, so a model would ideally be trained on this data. Therefore we do need to include them in our training set. 

In [ ]:
train_df[train_df.visit_number == 68031].head()

## Train a XGBoost Classifier for each time of day, and save the best model

The first step is to load a transformer for the ML training data to turn it into a format that our ML classifier can read. This is done using a function called create_column_transformer() which called ColumnTransfomer() a standard method in scikit-learn. This function could be changed for different input

The ColumnTransformer in scikit-learn is a tool that applies different transformations or preprocessing steps to different columns of a dataset in a single operation. OneHotEncoder converts categorical data into a format that can be provided to machine learning algorithms; without this, the model might interpret the categorical data as numerical, which would lead to incorrect results. With the OrdinalEncoder, categories are converted into ordered numerical values to reflect the inherent order in the age groups

We can also specify a grid of hyperparameters, so that the classifier will iterate though them to find the best fitting model. 

We are interested in predictions at different times of day. So we will train a model for each time of day. We will filter each visit so that it only appears once in the training data. A random number has already been included in the dataset to facilitate this.

We then iterate through the grid to find the best model for each time of day, keeping track of the best model and its results. 

The best model is saved, plus a dictionary of its metadata, including

* how many visits were in training, validation and test sets
* Area under ROC curve and log loss (performance metrics) for training (based on 5-fold cross validation), validation and test sets
* List of features and their importances in the model


#### Function for cross validation

The ML models will be trained across a range of different hyperparameter options. When evaluating the best model, we will save common ML metrics (AUC and logloss) and compare each model for the best (lowest) logloss. Apply a chronological approach to the cross-validation split is appropriate for tasks where the model needs to be validated on unseen, future data.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score, log_loss



def chronological_cross_validation(pipeline, X, y, n_splits=5):
    """
    Perform time series cross-validation.

    :param pipeline: The machine learning pipeline (preprocessing + model).
    :param X: Input features.
    :param y: Target variable.
    :param n_splits: Number of splits for cross-validation.
    :return: Dictionary with the average training and validation scores.
    """
    # Initialize TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Lists to collect scores for each fold
    train_aucs = []
    train_loglosses = []
    valid_aucs = []
    valid_loglosses = []

    # Iterate over train-test splits
    for train_index, test_index in tscv.split(X):
        X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]

        # Fit the pipeline to the training data
        # Note that you don't need to manually transform the data; the pipeline handles it
        pipeline.fit(X_train, y_train)
        
        # # To access transformed feature names:
        # transformed_cols = pipeline.named_steps['feature_transformer'].get_feature_names_out()
        # transformed_cols = [col.split('__')[-1] for col in transformed_cols]

        # Evaluate on the training split
        y_train_pred = pipeline.predict_proba(X_train)[:, 1]
        train_auc = roc_auc_score(y_train, y_train_pred)
        train_logloss = log_loss(y_train, y_train_pred)
        train_aucs.append(train_auc)
        train_loglosses.append(train_logloss)

        # Evaluate on the validation split
        y_valid_pred = pipeline.predict_proba(X_valid)[:, 1]
        valid_auc = roc_auc_score(y_valid, y_valid_pred)
        valid_logloss = log_loss(y_valid, y_valid_pred)
        valid_aucs.append(valid_auc)
        valid_loglosses.append(valid_logloss)

    # Calculate mean scores
    mean_train_auc = sum(train_aucs) / n_splits
    mean_train_logloss = sum(train_loglosses) / n_splits
    mean_valid_auc = sum(valid_aucs) / n_splits
    mean_valid_logloss = sum(valid_loglosses) / n_splits

    return {
        'train_auc': mean_train_auc,
        'valid_auc': mean_valid_auc,
        'train_logloss': mean_train_logloss,
        'valid_logloss': mean_valid_logloss,
    }


In [ ]:
# Initialise the model with given hyperparameters
def initialise_model(params):
    model = xgb.XGBClassifier(n_jobs=-1, use_label_encoder=False, eval_metric="logloss")
    model.set_params(**params)
    return model


# Set up the feature transformation
def create_column_transformer(df, ordinal_mappings=None):
    """
    Create a column transformer for a dataframe with dynamic column handling.

    :param df: Input dataframe.
    :param ordinal_mappings: A dictionary specifying the ordinal mappings for specific columns.
    :return: A configured ColumnTransformer object.
    """
    transformers = []
    
    # Default to an empty dict if no ordinal mappings are provided
    if ordinal_mappings is None:
        ordinal_mappings = {}

    for col in df.columns:
        if col in ordinal_mappings:
            # Ordinal encoding for specified columns with a predefined ordering
            transformers.append((col, OrdinalEncoder(categories=[ordinal_mappings[col]],
                                                     handle_unknown='use_encoded_value', unknown_value=np.nan), [col]))
        elif df[col].dtype == 'object' or (df[col].dtype == 'bool' or df[col].nunique() == 2):
        # OneHotEncoding for categorical or boolean columns
            transformers.append((col, OneHotEncoder(handle_unknown='ignore'), [col]))
        else:
            # Passthrough for other types of columns (e.g., numerical)
            transformers.append((col, 'passthrough', [col]))

    return ColumnTransformer(transformers)



In [ ]:

import xgboost as xgb

from sklearn.model_selection import ParameterGrid, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    log_loss,
    roc_auc_score,
)
from joblib import dump, load

from ed_admissions_utils import get_model_name, preprocess_data

# initialize a dict to save information about the best models for each time of day
best_model_results_dict = {}

# Specify where to save the dict and the trained models
model_file_path = PROJECT_ROOT / 'dissemination' / 'model-output' 

# Option to iterate through different hyperparameters for XGBoost
grid = {
    'n_estimators':[30, 40, 50],
    'subsample':[0.7,0.8,0.9],
    'colsample_bytree': [0.7,0.8,0.9]
}

# certain columns are not used in training
exclude_from_training_data = [
    "visit_number",
    "snapshot_datetime",
    "prediction_time"]


ordinal_mappings = {
    "age_group": ["0-17", "18-24", "25-34", "35-44", "45-54", "55-64", "65-74", "75-102"],
    "latest_acvpu": ["A", "C", "V", "P", "U"],
    "latest_manch_triage": ["Blue", "Green", "Yellow", "Orange", "Red"],
    "latest_pain_objective": ["Nil", "Mild", "Moderate", "Severe\\Very Severe"]
}


# Process each time of day
for prediction_time_ in prediction_times:

    print("\nProcessing :" + str(prediction_time_))

    # create a name for the model based on the time of day it is trained for
    MODEL__ED_ADMISSIONS__NAME = get_model_name(prediction_time_)

    # use this name in the path for saving best model
    full_path = model_file_path / MODEL__ED_ADMISSIONS__NAME 
    full_path = full_path.with_suffix('.joblib')

    # initialise data used for saving attributes of the model
    best_model_results_dict[MODEL__ED_ADMISSIONS__NAME] = {}
    best_valid_logloss = float('inf')
    results_dict = {}
    
    # get visits that were in at the time of day in question and preprocess the training, validation and test sets 
    X_train, y_train = preprocess_data(train_df, prediction_time_, exclude_from_training_data)
    X_valid, y_valid = preprocess_data(valid_df, prediction_time_, exclude_from_training_data)
    X_test, y_test = preprocess_data(test_df, prediction_time_, exclude_from_training_data)
    
    # save size of each set
    best_model_results_dict[MODEL__ED_ADMISSIONS__NAME]['train_valid_test_set_no'] = {
        'train_set_no' : len(X_train),
        'valid_set_no' : len(X_valid),
        'test_set_no' : len(X_test),
    }

    # iterate through the grid of hyperparameters
    for g in ParameterGrid(grid):
        model = initialise_model(g)
        
        # define a column transformer for the ordinal and categorical variables
        column_transformer = create_column_transformer(X_test)
        
        # create a pipeline with the feature transformer and the model
        pipeline = Pipeline([
            ('feature_transformer', column_transformer),
            ('classifier', m)
        ])

        # cross-validate on training set using the function created earlier
        cv_results = chronological_cross_validation(pipeline, X_train, y_train, n_splits=5)

        # Store results for this set of parameters in the results dictionary
        results_dict[str(g)] = {
            'train_auc': cv_results['train_auc'],
            'valid_auc': cv_results['valid_auc'],
            'train_logloss': cv_results['train_logloss'],
            'valid_logloss': cv_results['valid_logloss'],
        }
        
        # Update and save best model if current model is better on validation set
        if cv_results['valid_logloss'] < best_valid_logloss:

            # save the details of the best model
            best_model = str(g)
            best_valid_logloss = cv_results['valid_logloss']

            # save the best model params
            best_model_results_dict[MODEL__ED_ADMISSIONS__NAME]['best_params'] = str(g)

            # save the model metrics on training and validation set
            best_model_results_dict[MODEL__ED_ADMISSIONS__NAME]['train_valid_set_results'] = results_dict

            # score the model's performance on the test set  
            y_test_pred_proba = pipeline.predict_proba(X_test)[:, 1]
            test_auc = roc_auc_score(y_test, y_test_pred_proba)
            test_logloss = log_loss(y_test,y_test_pred_proba)
        
            best_model_results_dict[MODEL__ED_ADMISSIONS__NAME]['test_set_results'] = {
                'test_auc' : test_auc,
                'test_logloss' : test_logloss
            }

            # save the best features
            # To access transformed feature names:
            transformed_cols = pipeline.named_steps['feature_transformer'].get_feature_names_out()
            transformed_cols = [col.split('__')[-1] for col in transformed_cols]
            best_model_results_dict[MODEL__ED_ADMISSIONS__NAME]['best_model_features'] = {
                    'feature_names': transformed_cols,
                    'feature_importances': pipeline.named_steps['classifier'].feature_importances_.tolist()
                }

            # save the best model
            dump(pipeline, full_path)

# save the results dictionary      
filename_results_dict = 'best_model_results_dict.json'
full_path_results_dict = model_file_path / filename_results_dict

with open(full_path_results_dict, 'w') as f:
    json.dump(best_model_results_dict, f)  

In [ ]:
# with corrected code
for key, value in best_model_results_dict.items():
    print(f"Model: {key}; AUC: {round(value['test_set_results']['test_auc'],3)}; log loss {round(value['test_set_results']['test_logloss'],3)}")

In [ ]:
# before corrected code
for key, value in best_model_results_dict.items():
    print(f"Model: {key}; AUC: {round(value['test_set_results']['test_auc'],3)}; log loss {round(value['test_set_results']['test_logloss'],3)}")

In [ ]:
for key, value in best_model_results_dict.items():
    print(f"Model: {key}; AUC: {round(value['test_set_results']['test_auc'],3)}; log loss {round(value['test_set_results']['test_logloss'],3)}")